# 使用 Nsight Compute 优化在线 5G 机器学习

## 06 将向量存储在共享内存中

我们可以认识到，算法的数据输入向量在整个高斯循环中是相同的，并且基向量对于块中的每个样本都是相同的。这意味着“我们可以在同一块中的不同样本之间共享基（字典）向量”。我们假设向量的最大大小，并将它们缓存在 CUDA 的低延迟“共享内存”中。

使用共享内存的新版本已在代码中可用。要启用新版本，我们只需在 [apsm_versions.h](apsm/cpp/lib/apsm/apsm_versions.h) 的第 73 行中设置 `APSM_DETECT_VERSION` 标志。

打开 [apsm_detect.cu](apsm/cpp/lib/apsm/apsm_detect.cu) 的第 615 行以检查差异。您可能会注意到这里还有另一个中间步骤 `APSM_DETECT_SPLIT` 。对此内核的线性和高斯部分进行进一步分析并更好地了解更多时间花在哪里很有用。这对于本实验的进展并不重要，但如果您有兴趣，您可以在最后回到这里检查它。

这样，将 `APSM_DETECT_VERSION` 定义设置为 `apsm_version::APSM_DETECT_SHMEM` 后，使用以下命令重新编译代码并收集此新版本的分析器报告：

In [ ]:
%cd /dli/task/ncu/apsm/cpp/build
!make -j
!ncu -k kernel_apsm_detect --set full --import-source yes -f -o /dli/task/ncu/report_shmem \
    bin/APSM_tool -m QAM16 -s ../data/offline/rx/time/rxData_QAM16_alltx_converted.bin -r ../data/offline/tx/NOMA_signals_qam16_complex.bin

单击下方 `...` 以查看注释。

收集结果后，切换到带有 Nsight Compute 的 Ubuntu 实例并打开刚刚创建的报告文件 `/root/Desktop/reports/ncu/report_shmem.ncu-rep` 。为了便于比较，`将 SPB 步骤添加为新基线` 。只需将鼠标悬停在现有基线上并键入名称，即可为每个基线命名。（由于 CG 和 SPB 的性能非常相似，您也可以删除较旧的 CG 基线以使比较更易于阅读。只需单击相应基线名称旁边的彩色框即可）

在检查实际性能之前，我们可以在“启动统计信息”部分中验证内核现在是否确实在使用共享内存。它显示每个 CUDA 块配置了约 10KB 的静态共享内存。

<img src="images/ncu_report03_03.png">

现在，让我们确认我们的更改对内核运行时和指标产生了哪些影响。在详细信息页面上向上滚动以进行高级比较。

<img src="images/ncu_report03_01.png">

如您所见，新版本的速度提高了约 60%。

在“内存工作负载分析”部分，DRAM 的“内存吞吐量”显著增加。鉴于整个内存子系统利用率“内存吞吐量”在三个内核版本中并没有真正改变，这表明我们现在使用内存的效率要高得多。

造成这种情况的主要原因是“LG Throttle”停顿大大减少，正如在“Warp State Statistics”部分中看到的那样。我们记得这是步骤 5 中以前的 SPB 实现的主要问题。这一变化减轻了内存子系统的压力。

<img src="images/ncu_report03_02.png">

现在，虽然性能有所改善，LG Throttle 停顿也显著减少，但我们仍然看到导致相关延迟的停顿原因：`停顿屏障` 。让我们看看 [文档](https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#statistical-sampler) 中的描述：

*屏障*：Warp 在 CTA 屏障处等待兄弟 Warp 时停顿。大量 Warp 在屏障处等待通常是由于屏障前的代码路径发散造成的。这会导致一些 Warp 等待很长时间，直到其他 Warp 到达同步点。尽可能尝试将工作划分为统一的工作负载块。此外，尝试确定哪个屏障指令导致最多的停顿，并首先优化在该同步点之前执行的代码。

正如我们在顶部的 Speed Of Light 部分中看到的那样，Nsight Compute 仍然表明内核受到延迟限制，减少这些停顿可能会有益于内核性能。

如果时间不够，您可以直接转到 [总结](08_summary.ipynb)

如果您对其他步骤感兴趣，请在 [步骤 07](07_balanced.ipynb) 中继续进行优化